# Lab 1 - Creating the SQL Tables

In this lab, use `sqlalchemy` to create, populate, and query a table from the baseball database, as well as for the `super_hero_powers.csv` table.  

In [319]:
import pandas as pd
artwork = pd.read_csv("./data/Artworks.csv")

## Part 1 - Baseball Managers

In this part of the lab, you will walk through the process of creating a manager table from [Lahman’s Baseball Database](http://www.seanlahman.com/baseball-archive/statistics/)

## Task 1 - Download, unzip, rename 

1. Download the baseball database linked above (save to desktop)
2. Unzip the file and rename to `baseball`
3. Load the `core/Managers.csv` file into a pandas `DataFrame` using `read_csv`
4. Inspect the `column` names and `dtypes`

In [320]:
import pandas as pd
managers = pd.read_csv('~/Desktop/baseball/core/Managers.csv')
managers.head()

,playerID,yearID,teamID,lgID,inseason,G,W,L,rank,plyrMgr
0,wrighha01,1871,BS1,NaN,1,31,20,10,3.0,Y
1,woodji01,1871,CH1,NaN,1,28,19,9,2.0,Y
2,paborch01,1871,CL1,NaN,1,29,10,19,8.0,Y
3,lennobi01,1871,FW1,NaN,1,14,5,9,8.0,Y
4,deaneha01,1871,FW1,NaN,2,5,2,3,8.0,Y


In [321]:
managers.columns

Index(['playerID', 'yearID', 'teamID', 'lgID', 'inseason', 'G', 'W', 'L',
       'rank', 'plyrMgr'],
      dtype='object')

**Question:** Is there a candidate for a primary key?

In [322]:
[(col, managers[col].is_unique) for col in managers]

[('playerID', False),
 ('yearID', False),
 ('teamID', False),
 ('lgID', False),
 ('inseason', False),
 ('G', False),
 ('W', False),
 ('L', False),
 ('rank', False),
 ('plyrMgr', False)]

**Solution:** Add the `index` as an actual column

In [323]:
from dfply import mutate
managers = (managers >>
            mutate(id = managers.index))

In [324]:
managers.id.is_unique

True

In [325]:
managers.dtypes

playerID     object
yearID        int64
teamID       object
lgID         object
inseason      int64
G             int64
W             int64
L             int64
rank        float64
plyrMgr      object
id            int64
dtype: object

In [326]:
managers.shape

(3469, 11)

In [327]:
managers.head()

,playerID,yearID,teamID,lgID,inseason,G,W,L,rank,plyrMgr,id
0,wrighha01,1871,BS1,NaN,1,31,20,10,3.0,Y,0
1,woodji01,1871,CH1,NaN,1,28,19,9,2.0,Y,1
2,paborch01,1871,CL1,NaN,1,29,10,19,8.0,Y,2
3,lennobi01,1871,FW1,NaN,1,14,5,9,8.0,Y,3
4,deaneha01,1871,FW1,NaN,2,5,2,3,8.0,Y,4


#### Task 2 - Create a `sqlalchemy` types `dict`

In [328]:
from sqlalchemy import String, Integer
sql_types = {'id':Integer,
             'playerID':String, 
             'plyrMgr':String,
             'teamID':String, 
             'lgID':String, 
             'yearID':Integer, 
             'inseason':Integer, 
             'G':Integer, 
             'W':Integer, 
             'L':Integer,
             'rank':Integer}

#### Task 4 - Create an `engine` and `schema`

In [329]:
!rm databases/baseball.db

In [330]:
from sqlalchemy import create_engine
mang_eng = create_engine("sqlite:///databases/baseball.db")
mang_eng.echo = True
schema = pd.io.sql.get_schema(managers, 'manager', keys='id', con=mang_eng, dtype=sql_types)
print(schema)


CREATE TABLE manager (
	"playerID" VARCHAR, 
	"yearID" INTEGER, 
	"teamID" VARCHAR, 
	"lgID" VARCHAR, 
	inseason INTEGER, 
	"G" INTEGER, 
	"W" INTEGER, 
	"L" INTEGER, 
	rank INTEGER, 
	"plyrMgr" VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT manager_pk PRIMARY KEY (id)
)




#### Execute the `schema`

In [331]:
mang_eng.execute(schema)

2019-01-31 13:16:53,877 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-01-31 13:16:53,881 INFO sqlalchemy.engine.base.Engine ()
2019-01-31 13:16:53,884 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-01-31 13:16:53,885 INFO sqlalchemy.engine.base.Engine ()
2019-01-31 13:16:53,887 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE manager (
	"playerID" VARCHAR, 
	"yearID" INTEGER, 
	"teamID" VARCHAR, 
	"lgID" VARCHAR, 
	inseason INTEGER, 
	"G" INTEGER, 
	"W" INTEGER, 
	"L" INTEGER, 
	rank INTEGER, 
	"plyrMgr" VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT manager_pk PRIMARY KEY (id)
)


2019-01-31 13:16:53,889 INFO sqlalchemy.engine.base.Engine ()
2019-01-31 13:16:53,894 INFO sqlalchemy.engine.base.Engine COMMIT


#### Task 5 - Use `to_sql` with `if_exists='append'` to insert the data

In [332]:
managers.to_sql('manager', 
                con=mang_eng, 
                dtype=sql_types, 
                index=False,
                if_exists='append')

2019-01-31 13:16:53,912 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("manager")
2019-01-31 13:16:53,917 INFO sqlalchemy.engine.base.Engine ()
2019-01-31 13:16:53,924 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-31 13:16:54,729 INFO sqlalchemy.engine.base.Engine INSERT INTO manager ("playerID", "yearID", "teamID", "lgID", inseason, "G", "W", "L", rank, "plyrMgr", id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?,

2019-01-31 13:16:54,731 INFO sqlalchemy.engine.base.Engine ('wrighha01', 1871, 'BS1', None, 1, 31, 20, 10, 3.0, 'Y', 0, 'woodji01', 1871, 'CH1', None, 1, 28, 19, 9, 2.0, 'Y', 1, 'paborch01', 1871, 'CL1', None, 1, 29, 10, 19, 8.0, 'Y', 2, 'lennobi01', 1871, 'FW1', None, 1, 14, 5, 9, 8.0, 'Y', 3, 'deaneha01', 1871, 'FW1', None, 2, 5, 2, 3, 8.0, 'Y', 4, 'fergubo01', 1871, 'NY2', None, 1, 33, 16, 17, 5.0, 'Y', 5, 'mcbridi01', 1871, 'PH1', None, 1, 28, 21, 7, 1.0, 'Y', 6, 'hastisc01', 1871, 'RC1', None, 1, 25, 4, 21, 9.0, 'Y', 7, 'pikeli01', 1871, 'TRO', None, 1, 4, 1, 3, 6.0, 'Y', 8, 'cravebi01', 1871, 'TRO', None, 2, 25, 12, 12, 6.0, 'Y', 9, 'youngni99', 1871, 'WS3', None, 1, 32, 15, 15, 4.0, 'N', 10, 'cravebi01', 1872, 'BL1', None, 1, 41, 27, 13, 2.0, 'Y', 11, 'millsev01', 1872, 'BL1', None, 2, 17, 8, 6, 2.0, 'Y', 12, 'clintji01', 1872, 'BR1', None, 1, 11, 0, 11, 9.0, 'Y', 13, 'woodji01', 1872, 'BR1', None, 2, 18, 3, 15, 9.0, 'Y', 14, 'fergubo01', 1872, 'BR2', None, 1, 37, 9, 28, 7.0, 'Y

2019-01-31 13:16:54,795 INFO sqlalchemy.engine.base.Engine COMMIT


#### Task 6 - Query the table to make sure it all worked

In [333]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select

mang_eng2 = create_engine("sqlite:///databases/baseball.db") 
Session = sessionmaker(mang_eng)
session = Session()

In [334]:
Base = automap_base()
Base.prepare(mang_eng2, reflect=True)
Manager = Base.classes.manager

In [335]:
from more_sqlalchemy import result_dicts
stmt = select('*').select_from(Manager)
session.execute(stmt).fetchmany(5) >> result_dicts

2019-01-31 13:16:54,837 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-31 13:16:54,838 INFO sqlalchemy.engine.base.Engine SELECT * 
FROM manager
2019-01-31 13:16:54,840 INFO sqlalchemy.engine.base.Engine ()


[{'playerID': 'wrighha01',
  'yearID': 1871,
  'teamID': 'BS1',
  'lgID': None,
  'inseason': 1,
  'G': 31,
  'W': 20,
  'L': 10,
  'rank': 3,
  'plyrMgr': 'Y',
  'id': 0},
 {'playerID': 'woodji01',
  'yearID': 1871,
  'teamID': 'CH1',
  'lgID': None,
  'inseason': 1,
  'G': 28,
  'W': 19,
  'L': 9,
  'rank': 2,
  'plyrMgr': 'Y',
  'id': 1},
 {'playerID': 'paborch01',
  'yearID': 1871,
  'teamID': 'CL1',
  'lgID': None,
  'inseason': 1,
  'G': 29,
  'W': 10,
  'L': 19,
  'rank': 8,
  'plyrMgr': 'Y',
  'id': 2},
 {'playerID': 'lennobi01',
  'yearID': 1871,
  'teamID': 'FW1',
  'lgID': None,
  'inseason': 1,
  'G': 14,
  'W': 5,
  'L': 9,
  'rank': 8,
  'plyrMgr': 'Y',
  'id': 3},
 {'playerID': 'deaneha01',
  'yearID': 1871,
  'teamID': 'FW1',
  'lgID': None,
  'inseason': 2,
  'G': 5,
  'W': 2,
  'L': 3,
  'rank': 8,
  'plyrMgr': 'Y',
  'id': 4}]

## Part 2 - Awards for Managers

Now add a table for the `AwardsManagers.csv` table.

In [336]:
import pandas as pd
Awmanagers = pd.read_csv('~/Desktop/baseball/core/AwardsManagers.csv')
Awmanagers.head()

,playerID,awardID,yearID,lgID,tie,notes
0,larusto01,BBWAA Manager of the Year,1983,AL,NaN,NaN
1,lasorto01,BBWAA Manager of the Year,1983,NL,NaN,NaN
2,andersp01,BBWAA Manager of the Year,1984,AL,NaN,NaN
3,freyji99,BBWAA Manager of the Year,1984,NL,NaN,NaN
4,coxbo01,BBWAA Manager of the Year,1985,AL,NaN,NaN


In [337]:
Awmanagers.columns

Index(['playerID', 'awardID', 'yearID', 'lgID', 'tie', 'notes'], dtype='object')

In [338]:
[(col, Awmanagers[col].is_unique) for col in Awmanagers]

[('playerID', False),
 ('awardID', False),
 ('yearID', False),
 ('lgID', False),
 ('tie', False),
 ('notes', False)]

In [339]:
from dfply import mutate
Awmanagers = (Awmanagers >>
            mutate(id = Awmanagers.index))

In [340]:
Awmanagers.id.is_unique

True

In [341]:
Awmanagers.dtypes

playerID    object
awardID     object
yearID       int64
lgID        object
tie         object
notes       object
id           int64
dtype: object

In [342]:
Awmanagers.shape

(179, 7)

In [343]:
Awmanagers.head()

,playerID,awardID,yearID,lgID,tie,notes,id
0,larusto01,BBWAA Manager of the Year,1983,AL,NaN,NaN,0
1,lasorto01,BBWAA Manager of the Year,1983,NL,NaN,NaN,1
2,andersp01,BBWAA Manager of the Year,1984,AL,NaN,NaN,2
3,freyji99,BBWAA Manager of the Year,1984,NL,NaN,NaN,3
4,coxbo01,BBWAA Manager of the Year,1985,AL,NaN,NaN,4


In [344]:
from sqlalchemy import String, Integer
sql_types = {'playerID':String, 
             'awardID':String,  
             'yearID':Integer,
             'lgID':String,
             'tie': String,
             'notes':String,
             'id':Integer
            }

In [345]:
from sqlalchemy import create_engine
mang_eng.echo = True
schema = pd.io.sql.get_schema(Awmanagers, 'AwardManager', keys='id', con=mang_eng, dtype=sql_types)
print(schema)


CREATE TABLE "AwardManager" (
	"playerID" VARCHAR, 
	"awardID" VARCHAR, 
	"yearID" INTEGER, 
	"lgID" VARCHAR, 
	tie VARCHAR, 
	notes VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT "AwardManager_pk" PRIMARY KEY (id)
)




In [346]:
mang_eng.execute(schema)

2019-01-31 13:16:54,976 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "AwardManager" (
	"playerID" VARCHAR, 
	"awardID" VARCHAR, 
	"yearID" INTEGER, 
	"lgID" VARCHAR, 
	tie VARCHAR, 
	notes VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT "AwardManager_pk" PRIMARY KEY (id)
)


2019-01-31 13:16:54,978 INFO sqlalchemy.engine.base.Engine ()
2019-01-31 13:16:54,981 INFO sqlalchemy.engine.base.Engine COMMIT


In [347]:
Awmanagers.to_sql('AwardManager', 
                con=mang_eng, 
                dtype=sql_types, 
                index=False,
                if_exists='append')

2019-01-31 13:16:54,997 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("AwardManager")
2019-01-31 13:16:54,998 INFO sqlalchemy.engine.base.Engine ()
2019-01-31 13:16:55,003 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-31 13:16:55,018 INFO sqlalchemy.engine.base.Engine INSERT INTO "AwardManager" ("playerID", "awardID", "yearID", "lgID", tie, notes, id) VALUES (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?

2019-01-31 13:16:55,025 INFO sqlalchemy.engine.base.Engine COMMIT
2019-01-31 13:16:55,029 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-01-31 13:16:55,030 INFO sqlalchemy.engine.base.Engine ()


In [348]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select

mang_eng2 = create_engine("sqlite:///databases/baseball.db") 
Session = sessionmaker(mang_eng)
session = Session()

In [349]:
Base = automap_base()
Base.prepare(mang_eng2, reflect=True)
AwardManager = Base.classes.AwardManager

In [350]:
from more_sqlalchemy import result_dicts
stmt = select('*').select_from(AwardManager)
session.execute(stmt).fetchmany(5) >> result_dicts

2019-01-31 13:16:55,068 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-31 13:16:55,070 INFO sqlalchemy.engine.base.Engine SELECT * 
FROM "AwardManager"
2019-01-31 13:16:55,071 INFO sqlalchemy.engine.base.Engine ()


[{'playerID': 'larusto01',
  'awardID': 'BBWAA Manager of the Year',
  'yearID': 1983,
  'lgID': 'AL',
  'tie': None,
  'notes': None,
  'id': 0},
 {'playerID': 'lasorto01',
  'awardID': 'BBWAA Manager of the Year',
  'yearID': 1983,
  'lgID': 'NL',
  'tie': None,
  'notes': None,
  'id': 1},
 {'playerID': 'andersp01',
  'awardID': 'BBWAA Manager of the Year',
  'yearID': 1984,
  'lgID': 'AL',
  'tie': None,
  'notes': None,
  'id': 2},
 {'playerID': 'freyji99',
  'awardID': 'BBWAA Manager of the Year',
  'yearID': 1984,
  'lgID': 'NL',
  'tie': None,
  'notes': None,
  'id': 3},
 {'playerID': 'coxbo01',
  'awardID': 'BBWAA Manager of the Year',
  'yearID': 1985,
  'lgID': 'AL',
  'tie': None,
  'notes': None,
  'id': 4}]

## Part 3 - Super Hero Powers

Now make a database and table for the super hero powers.

## Problem 1
    
**Task:** One the `super_hero_powers.csv` and verify that the contents of the columns are all Boolean.  In this problem, you need to

1. Create a `dict` that defines the `pandas` column type
2. Read the file in using a `pd.read_csv`.
3. Clean up all the column labels.
    
**Be sure to write clean code!**


In [351]:
import pandas as pd
powers = pd.read_csv('~/Desktop/DSCI_430/DSCI430/data/super_hero_powers.csv')
powers.head()

,hero_names,Agility,Accelerated Healing,Lantern Power Ring,Dimensional Awareness,Cold Resistance,Durability,Stealth,Energy Absorption,Flight,...,Web Creation,Reality Warping,Odin Force,Symbiote Costume,Speed Force,Phoenix Force,Molecular Dissipation,Vision - Cryo,Omnipresent,Omniscient
0,3-D Man,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,True,True,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [352]:
from dfply import mutate
powers = (powers >>
            mutate(id = powers.index))

In [353]:
powers.head()

,hero_names,Agility,Accelerated Healing,Lantern Power Ring,Dimensional Awareness,Cold Resistance,Durability,Stealth,Energy Absorption,Flight,...,Reality Warping,Odin Force,Symbiote Costume,Speed Force,Phoenix Force,Molecular Dissipation,Vision - Cryo,Omnipresent,Omniscient,id
0,3-D Man,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
1,A-Bomb,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,1
2,Abe Sapien,True,True,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,2
3,Abin Sur,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,3
4,Abomination,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,4


In [354]:
powers.dtypes

hero_names               object
Agility                    bool
Accelerated Healing        bool
Lantern Power Ring         bool
Dimensional Awareness      bool
Cold Resistance            bool
Durability                 bool
Stealth                    bool
Energy Absorption          bool
Flight                     bool
Danger Sense               bool
Underwater breathing       bool
Marksmanship               bool
Weapons Master             bool
Power Augmentation         bool
Animal Attributes          bool
Longevity                  bool
Intelligence               bool
Super Strength             bool
Cryokinesis                bool
Telepathy                  bool
Energy Armor               bool
Energy Blasts              bool
Duplication                bool
Size Changing              bool
Density Control            bool
Stamina                    bool
Astral Travel              bool
Audio Control              bool
Dexterity                  bool
                          ...  
Vision -

In [355]:
powers.shape

(667, 169)

In [356]:
powers.columns

Index(['hero_names', 'Agility', 'Accelerated Healing', 'Lantern Power Ring',
       'Dimensional Awareness', 'Cold Resistance', 'Durability', 'Stealth',
       'Energy Absorption', 'Flight',
       ...
       'Reality Warping', 'Odin Force', 'Symbiote Costume', 'Speed Force',
       'Phoenix Force', 'Molecular Dissipation', 'Vision - Cryo',
       'Omnipresent', 'Omniscient', 'id'],
      dtype='object', length=169)

In [357]:
from sqlalchemy import String, Integer,Boolean
clean_col = lambda col: col.replace(" ", "_").replace("(", "").replace(")", "").replace(".", "").lower()
def type_col(col):
    if col =="hero_names": 
        return String
    elif col =="id": 
        return Integer
    else: 
        return Boolean
powers.columns = powers.columns.str.replace(" ", "_").str.lower()
sql_types = {clean_col(col):type_col(col) for col in powers.columns}
sql_types

{'hero_names': sqlalchemy.sql.sqltypes.String,
 'agility': sqlalchemy.sql.sqltypes.Boolean,
 'accelerated_healing': sqlalchemy.sql.sqltypes.Boolean,
 'lantern_power_ring': sqlalchemy.sql.sqltypes.Boolean,
 'dimensional_awareness': sqlalchemy.sql.sqltypes.Boolean,
 'cold_resistance': sqlalchemy.sql.sqltypes.Boolean,
 'durability': sqlalchemy.sql.sqltypes.Boolean,
 'stealth': sqlalchemy.sql.sqltypes.Boolean,
 'energy_absorption': sqlalchemy.sql.sqltypes.Boolean,
 'flight': sqlalchemy.sql.sqltypes.Boolean,
 'danger_sense': sqlalchemy.sql.sqltypes.Boolean,
 'underwater_breathing': sqlalchemy.sql.sqltypes.Boolean,
 'marksmanship': sqlalchemy.sql.sqltypes.Boolean,
 'weapons_master': sqlalchemy.sql.sqltypes.Boolean,
 'power_augmentation': sqlalchemy.sql.sqltypes.Boolean,
 'animal_attributes': sqlalchemy.sql.sqltypes.Boolean,
 'longevity': sqlalchemy.sql.sqltypes.Boolean,
 'intelligence': sqlalchemy.sql.sqltypes.Boolean,
 'super_strength': sqlalchemy.sql.sqltypes.Boolean,
 'cryokinesis': sqlal

In [358]:
from dfply import rename, head
clean_col = lambda col: col.replace(" ", "_").replace("(", "").replace(")", "").replace(".", "").lower()
(powers>>
    rename(**{clean_col(origin_col):origin_col
             for origin_col in powers.columns})>>
          head)

,hero_names,agility,accelerated_healing,lantern_power_ring,dimensional_awareness,cold_resistance,durability,stealth,energy_absorption,flight,...,reality_warping,odin_force,symbiote_costume,speed_force,phoenix_force,molecular_dissipation,vision_-_cryo,omnipresent,omniscient,id
0,3-D Man,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
1,A-Bomb,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,1
2,Abe Sapien,True,True,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,2
3,Abin Sur,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,3
4,Abomination,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,4


In [359]:
powers.columns

Index(['hero_names', 'agility', 'accelerated_healing', 'lantern_power_ring',
       'dimensional_awareness', 'cold_resistance', 'durability', 'stealth',
       'energy_absorption', 'flight',
       ...
       'reality_warping', 'odin_force', 'symbiote_costume', 'speed_force',
       'phoenix_force', 'molecular_dissipation', 'vision_-_cryo',
       'omnipresent', 'omniscient', 'id'],
      dtype='object', length=169)

## Problem 2
    
Now define an `sqlalchemy` table for these data using `pandas` `to_sql` dataframe method.  You can use the `sqlalchemy.String` and `sqlalchemy.Boolean` columns type, which are [documented here](https://docs.sqlalchemy.org/en/latest/core/type_basics.html)

In [360]:
from sqlalchemy import create_engine
heros_eng = create_engine("sqlite:///databases/heros.db")
heros_eng.echo = True
schema = pd.io.sql.get_schema(powers, 'powers_tbl', keys='id', con=heros_eng, dtype=sql_types)
print(schema)


CREATE TABLE powers_tbl (
	hero_names VARCHAR, 
	agility BOOLEAN, 
	accelerated_healing BOOLEAN, 
	lantern_power_ring BOOLEAN, 
	dimensional_awareness BOOLEAN, 
	cold_resistance BOOLEAN, 
	durability BOOLEAN, 
	stealth BOOLEAN, 
	energy_absorption BOOLEAN, 
	flight BOOLEAN, 
	danger_sense BOOLEAN, 
	underwater_breathing BOOLEAN, 
	marksmanship BOOLEAN, 
	weapons_master BOOLEAN, 
	power_augmentation BOOLEAN, 
	animal_attributes BOOLEAN, 
	longevity BOOLEAN, 
	intelligence BOOLEAN, 
	super_strength BOOLEAN, 
	cryokinesis BOOLEAN, 
	telepathy BOOLEAN, 
	energy_armor BOOLEAN, 
	energy_blasts BOOLEAN, 
	duplication BOOLEAN, 
	size_changing BOOLEAN, 
	density_control BOOLEAN, 
	stamina BOOLEAN, 
	astral_travel BOOLEAN, 
	audio_control BOOLEAN, 
	dexterity BOOLEAN, 
	omnitrix BOOLEAN, 
	super_speed BOOLEAN, 
	possession BOOLEAN, 
	animal_oriented_powers BOOLEAN, 
	"weapon-based_powers" BOOLEAN, 
	electrokinesis BOOLEAN, 
	darkforce_manipulation BOOLEAN, 
	death_touch BOOLEAN, 
	teleportation

In [361]:
#heros_eng.execute(schema)

2019-01-31 13:16:55,481 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-01-31 13:16:55,483 INFO sqlalchemy.engine.base.Engine ()
2019-01-31 13:16:55,485 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-01-31 13:16:55,487 INFO sqlalchemy.engine.base.Engine ()
2019-01-31 13:16:55,489 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE powers_tbl (
	hero_names VARCHAR, 
	agility BOOLEAN, 
	accelerated_healing BOOLEAN, 
	lantern_power_ring BOOLEAN, 
	dimensional_awareness BOOLEAN, 
	cold_resistance BOOLEAN, 
	durability BOOLEAN, 
	stealth BOOLEAN, 
	energy_absorption BOOLEAN, 
	flight BOOLEAN, 
	danger_sense BOOLEAN, 
	underwater_breathing BOOLEAN, 
	marksmanship BOOLEAN, 
	weapons_master BOOLEAN, 
	power_augmentation BOOLEAN, 
	animal_attributes BOOLEAN, 
	longevity BOOLEAN, 
	intelligence BOOLEAN, 
	super_strength BOOLEAN, 
	cryokinesis BOOLEAN, 
	telepathy BOOLEAN, 
	energy_armor BOOLEAN,

2019-01-31 13:16:55,491 INFO sqlalchemy.engine.base.Engine ()
2019-01-31 13:16:55,497 INFO sqlalchemy.engine.base.Engine COMMIT


In [362]:
powers.to_sql('powers_tbl', 
                con=heros_eng, 
                dtype=sql_types, 
                index=False,
                if_exists='append')

2019-01-31 13:16:55,587 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("powers_tbl")
2019-01-31 13:16:55,589 INFO sqlalchemy.engine.base.Engine ()
2019-01-31 13:16:55,596 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-31 13:16:57,795 INFO sqlalchemy.engine.base.Engine INSERT INTO powers_tbl (hero_names, agility, accelerated_healing, lantern_power_ring, dimensional_awareness, cold_resistance, durability, stealth, energy_absorption, flight, danger_sense, underwater_breathing, marksmanship, weapons_master, power_augmentation, animal_attributes, longevity, intelligence, super_strength, cryokinesis, telepathy, energy_armor, energy_blasts, duplication, size_changing, density_control, stamina, astral_travel, audio_control, dexterity, omnitrix, super_speed, possession, animal_oriented_powers, "weapon-based_powers", electrokinesis, darkforce_manipulation, death_touch, teleportation, enhanced_senses, telekinesis, energy_beams, magic, hyperkinesis, jump, clairvoyance, dimension

2019-01-31 13:16:57,803 INFO sqlalchemy.engine.base.Engine ('3-D Man', 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'A-Bomb', 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

2019-01-31 13:16:57,957 INFO sqlalchemy.engine.base.Engine COMMIT


## Problem 3
    
Now you need to make a new `engine`, `inspect` your database, and make a `session` to query the database.

In [363]:
from sqlalchemy import inspect
#insp = inspect(powers_tbl)

In [364]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select

Session = sessionmaker(heros_eng)
session = Session()

In [365]:
Base = automap_base()
Base.prepare(heros_eng, reflect=True)
Powers = Base.classes.powers_tbl

2019-01-31 13:16:57,986 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-01-31 13:16:57,986 INFO sqlalchemy.engine.base.Engine ()
2019-01-31 13:16:57,989 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("powers_tbl")
2019-01-31 13:16:57,991 INFO sqlalchemy.engine.base.Engine ()
2019-01-31 13:16:58,034 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'powers_tbl' AND type = 'table'
2019-01-31 13:16:58,035 INFO sqlalchemy.engine.base.Engine ()
2019-01-31 13:16:58,040 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("powers_tbl")
2019-01-31 13:16:58,041 INFO sqlalchemy.engine.base.Engine ()
2019-01-31 13:16:58,043 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'powers_tbl' AND type = 'table'
2019-01-31 13:16:58,044 INFO sqlalchemy.engine.ba

## Problem 4
    
Perform `sqlalchemy` queries to answer each of the following questions.

1. How many heroes have both Super Strength and Super Speed?
2. How many heroes have names that start with the word *Black*
3. Are heroes with Agility more likely to have Stealth?
4. What fraction of all heroes that can fly also have Super Strength?
5. Consider heroes that have names that contain `"girl"`, `"boy"`, `"woman"`, or `"man"`.  Compute the following ratio

$$\frac{N(\text{boy or man})}{N(\text{girl or woman}}$$

**Hint:** You will need to use some combination of `where`, `group_by`, and `count` for each part.

In [465]:
from more_sqlalchemy import result_dicts
from sqlalchemy import select, func, and_, or_
stmt = select([func.count()]).select_from(Powers)
session.execute(stmt).fetchmany(5) >> result_dicts

2019-01-31 17:13:43,853 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM powers_tbl
2019-01-31 17:13:43,855 INFO sqlalchemy.engine.base.Engine ()


[{'count_1': 667}]

In [377]:
###Number 1
num_one = select([func.count()]).where(and_(Powers.super_strength == True, Powers.super_speed ==True))
session.execute(num_one).fetchmany(5) >> result_dicts

2019-01-31 13:30:12,826 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM powers_tbl 
WHERE powers_tbl.super_strength = 1 AND powers_tbl.super_speed = 1
2019-01-31 13:30:12,827 INFO sqlalchemy.engine.base.Engine ()


[{'count_1': 219}]

In [379]:
###Number 2
num_two = select([func.count()]).where(Powers.hero_names.like("Black%"))
session.execute(num_two).fetchmany(5) >> result_dicts

2019-01-31 13:35:50,364 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM powers_tbl 
WHERE powers_tbl.hero_names LIKE ?
2019-01-31 13:35:50,365 INFO sqlalchemy.engine.base.Engine ('Black%',)


[{'count_1': 16}]

In [436]:
###Number 3
find_agility_stealth = select([func.count(),Powers.agility, Powers.stealth]).\
                group_by(Powers.agility).\
                group_by(Powers.stealth)
result = session.execute(find_agility_stealth).fetchall()>> result_dicts
result[0]['agility']
has_agility = [(elem['count_1'], elem['stealth']) for elem in result if elem['agility']]
has_agility[1][0] / (has_agility[0][0] + has_agility[1][0])

2019-01-31 16:45:43,351 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1, powers_tbl.agility, powers_tbl.stealth 
FROM powers_tbl GROUP BY powers_tbl.agility, powers_tbl.stealth
2019-01-31 16:45:43,352 INFO sqlalchemy.engine.base.Engine ()


0.3925619834710744

In [435]:
no_agility = [(elem['count_1'], elem['stealth']) for elem in result if not elem['agility']]
no_agility[1][0] / (no_agility[0][0] + no_agility[1][0])

0.07294117647058823

In [454]:
###Number 4
find_fly_strength = select([func.count(),Powers.flight, Powers.super_strength]).\
                    where(Powers.flight == True).\
                    group_by(Powers.flight).\
                    group_by(Powers.super_strength)
result= session.execute(find_fly_strength).fetchall()>> result_dicts
result
flight_info = [elem['count_1'] for elem in result]
strength_fly_ratio = flight_info[1]/(flight_info[0]+flight_info[1])
strength_fly_ratio

2019-01-31 16:57:18,703 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1, powers_tbl.flight, powers_tbl.super_strength 
FROM powers_tbl 
WHERE powers_tbl.flight = 1 GROUP BY powers_tbl.flight, powers_tbl.super_strength
2019-01-31 16:57:18,704 INFO sqlalchemy.engine.base.Engine ()


0.6933962264150944

In [479]:
###Number 5
find_female = select([func.count()]).\
            where(or_(Powers.hero_names.ilike("%girl%"), Powers.hero_names.ilike("%woman%")))
female_result= session.execute(find_female).fetchall()>> result_dicts
access_females= [elem['count_1'] for elem in female_result]
access_females[0]

2019-01-31 21:55:03,447 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM powers_tbl 
WHERE lower(powers_tbl.hero_names) LIKE lower(?) OR lower(powers_tbl.hero_names) LIKE lower(?)
2019-01-31 21:55:03,450 INFO sqlalchemy.engine.base.Engine ('%girl%', '%woman%')


26

In [480]:
find_male = select([func.count()]).\
            where(or_(Powers.hero_names.ilike("%boy%"), Powers.hero_names.ilike("%man%")))
male_result= session.execute(find_male).fetchall()>> result_dicts
access_males= [elem['count_1'] for elem in male_result]
access_males[0]

2019-01-31 21:58:10,512 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM powers_tbl 
WHERE lower(powers_tbl.hero_names) LIKE lower(?) OR lower(powers_tbl.hero_names) LIKE lower(?)
2019-01-31 21:58:10,514 INFO sqlalchemy.engine.base.Engine ('%boy%', '%man%')


62

In [482]:
male_female_ratio = access_males[0]/access_females[0]
male_female_ratio

2.3846153846153846

## Problem 5

Tell me another cool fact about the super powers.